In [1]:
from __future__ import division, print_function
import matplotlib
import matplotlib.pyplot as plt
from IPython.display import Image, display, clear_output
import numpy as np
import matplotlib.pyplot as plt
import sklearn.datasets
import tensorflow as tf
import math
from tensorflow.python.framework.ops import reset_default_graph

from sklearn.utils import shuffle

In [8]:

# To speed up training we'll only work on a subset of the data containing only the numbers 0, 1, 4, 9.
# We discretize the data to 0 and 1 in order to use it with a 
# bernoulli observation model p(x|z) = Ber(mu(z))

def bernoulli_sample(x):
    return np.random.binomial(1,x,size=x.shape).astype('float32')

# Load data from compressed file with mnist
data = np.load('data/mnist.npz')

gpu_opts = tf.GPUOptions(per_process_gpu_memory_fraction=0.45)

# Possible classes
classes = list(range(10))

# Set the classes we want to use.
included_classes = [0, 1, 4, 9] 

idxs_train = []
idxs_valid = []
idxs_test = []
num_classes = 0
for c in included_classes:
    if c in classes:
        num_classes += 1
        idxs_train += np.where(data['y_train'] == c)[0].tolist()
        idxs_valid += np.where(data['y_valid'] == c)[0].tolist()
        idxs_test += np.where(data['y_test'] == c)[0].tolist()

print("Number of classes included:", num_classes)
x_train = bernoulli_sample(data['X_train'][idxs_train]).astype('float32')
# Since this is unsupervised, the targets are only used for validation.
targets_train = data['y_train'][idxs_train].astype('int32')
x_train, targets_train = shuffle(x_train, targets_train, random_state=1234)

x_valid = bernoulli_sample(data['X_valid'][idxs_valid]).astype('float32')
targets_valid = data['y_valid'][idxs_valid].astype('int32')

x_test = bernoulli_sample(data['X_test'][idxs_test]).astype('float32')
targets_test = data['y_test'][idxs_test].astype('int32')

print("training set dim(%i, %i)." % x_train.shape)
print("validation set dim(%i, %i)." % x_valid.shape)
print("test set dim(%i, %i)." % x_test.shape)

Number of classes included: 4
training set dim(20457, 784).
validation set dim(3999, 784).
test set dim(4106, 784).


In [9]:
from tensorflow import layers
from tensorflow.contrib.layers import fully_connected, convolution2d, convolution2d_transpose, batch_norm, max_pool2d, dropout
from tensorflow.python.ops.nn import relu, elu, relu6, sigmoid, tanh, softmax, softplus

# Error functions    
def sum_of_squared_errors(p, t):
    return tf.reduce_sum(tf.square(p - t), axis=[1])


# computing cross entropy per sample
def categorical_cross_entropy(p, t, eps=1e-10):
    return -tf.reduce_sum(t * tf.log(p+eps), axis=[1])


def binary_cross_entropy(p, t, eps=1e-10):
    return -tf.reduce_sum(t * tf.log(p+eps) + (1-t) * tf.log(1-p+eps), axis=-1)


def kl_normal2_stdnormal(mean, log_var, eps=0.0):
    return -0.5 * tf.reduce_sum(1 + log_var - tf.square(mean) - tf.exp(log_var), axis=1)


c = - 0.5 * math.log(2*math.pi)
def log_normal2(x, mean, log_var, eps=0.0):
    return tf.reduce_sum(c - log_var/2 - tf.square(x - mean) / (2 * tf.exp(log_var) + eps), axis=[1])

In [24]:
# reset graph
reset_default_graph()

# -- THE MODEL --#
num_channels = 1; #Black and white for MNIST
num_classes = len(included_classes)
k = 16;
height = width = 28

# Layer definitions
def layer(x, units):
    x = fully_connected(x, num_outputs=units, activation_fn=relu,
                         normalizer_fn=batch_norm)
    x = convolution2d(x, num_outputs=units, kernel_size=(3, 3),
                             stride=1)
    return dropout(x, is_training=is_training_pl)
    
def dense_block(x, num_layers):
    res = []
    for i in range(num_layers):
        layer_output = layer(x, k)
        x = tf.concat([x, layer_output], axis=-1)
        res.append(layer_output)
    return x, res
    

def transition_up(x, units):
    return convolution2d_transpose(x, num_outputs=units, kernel_size=(3, 3), stride=2)
    
    
def transition_down(x, units, pooling=True):
    #x = batch_norm(x, scope='tdown_batch_norm') #Batch norm should be included in fully_connected layer below
    x = fully_connected(x, num_outputs=units, activation_fn=relu,
                     normalizer_fn=batch_norm)
    x = convolution2d(x, num_outputs=units, kernel_size=(1, 1),
                         stride=1)
    x = dropout(x, is_training=is_training_pl)
    if pooling:
        x = max_pool2d(x, kernel_size=(2, 2))
    return x

# - Tiramisu Architecture - #
# Input placeholder
x_pl = tf.placeholder(tf.float32, [None, height, width, num_channels], 'x_pl')
y_pl = tf.placeholder(tf.float32, [None, height, width, num_classes], 'y_pl')
is_training_pl = tf.placeholder(tf.bool, name="is-training_pl")
print('x_pl', x_pl.shape)
print('y_pl', y_pl.shape)

with tf.name_scope('tiramisu'):
    # DOWN SAMPLING
    x = convolution2d(x_pl, num_outputs=k, kernel_size=(3, 3),
                             stride=1, scope="pre-convolution")
    print(x.shape)
    skip1 = dense_block(x, 4)[0]
    
    skip1 = tf.concat([x, skip1], axis=-1)
    x = transition_down(skip1, 4*16, True)
    print(x.shape)

    skip2 = dense_block(x, 5)[0]
    skip2 = tf.concat([x, skip2], axis=-1)
    x = transition_down(skip2, 5*16+4*16, True)
    print(x.shape)

    # BOTTLENECK
    x = dense_block(x, 15)[0]
    print(x.shape)

    # UPSAMPLING
    x = transition_up(x, 15*16)
    x = tf.concat([x, skip2], axis=-1)
    x = dense_block(x, 5)[0]
    print(x.shape)

    x = transition_up(x, 5*16)
    skipUp = tf.concat([x, skip1], axis=-1)
    x = dense_block(skipUp, 4)[0]
    x = tf.concat([x, skipUp], axis=-1)
    print(x.shape)

    # Output layers
    x = convolution2d(x, num_outputs=num_classes, kernel_size=(1, 1),
                             stride=1, scope="post-convolution")
    print(x.shape)
    y = fully_connected(x, num_outputs=num_classes, activation_fn=softmax, scope="SoftMax")
    print(y.shape)

print("Model built")

x_pl (?, 28, 28, 1)
y_pl (?, 28, 28, 4)
(?, 28, 28, 16)
(?, 14, 14, 64)
(?, 7, 7, 144)
(?, 7, 7, 384)
(?, 14, 14, 528)
(?, 28, 28, 416)
(?, 28, 28, 4)
(?, 28, 28, 4)
Model built


In [25]:
with tf.variable_scope('loss'):
    # computing cross entropy per sample
    cross_entropy = -tf.reduce_sum(y_pl * tf.log(y+1e-8), reduction_indices=[1])

    # averaging over samples
    cross_entropy = tf.reduce_mean(cross_entropy)

    
with tf.variable_scope('training'):
    # defining our optimizer
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001)

    # applying the gradients
    train_op = optimizer.minimize(cross_entropy)

    
with tf.variable_scope('performance'):
    # making a one-hot encoded vector of correct (1) and incorrect (0) predictions
    correct_prediction = tf.equal(tf.argmax(y, axis=1), tf.argmax(y_pl, axis=1))

    # averaging the one-hot encoded vector
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


In [26]:
# test the forward pass
batch_size = 32
x_shape = [batch_size]
x_shape.extend(x_pl.get_shape().as_list()[1:])
_x_test = np.zeros(shape=x_shape)
y_shape = [batch_size]
y_shape.extend(y_pl.get_shape().as_list()[1:])
_y_test = np.zeros(shape=y_shape)
# initialize the Session
with tf.Session(config=tf.ConfigProto(gpu_options=gpu_opts)) as sess:
    sess.run(tf.global_variables_initializer())
    feed_dict = {x_pl: _x_test, is_training_pl : True}
    y_pred = sess.run(fetches=y, feed_dict=feed_dict)
    
assert y_pred.shape == _y_test.shape, "ERROR the output shape is not as expected!" \
        + " Output shape should be " + str(_y_test.shape) + ' but was ' + str(y_pred.shape)

print('Forward pass successful!')

Forward pass successful!
